# 0. Load Env

In [37]:
import json
import pandas as pd
from typing import List, Dict, Tuple, Any, Callable, Union
from functools import reduce
from operator import getitem
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
pd.set_option('display.max_colwidth', None)

In [38]:
TWEET_DICT_KEY_MAPPING = [
    (("url",), "url",),
    (("content",), "content",),
    (("id",), "tweet_id",),
    (("user", "id",), "user_id",),
    (("user", "username",), "user_name",),
    (("date",), "datetime",),
]

In [39]:
def nested_dictionary_get(d: Dict, key: Tuple):
    return reduce(getitem, key, d)


def parse_and_reduce_tweet(tweet: Dict) -> Dict[str, Any]:
    """
    Parses a tweet (represented as a dictionary), reducing the tweet to the schema defined
    in `keys_set`
    """
    new_dict = {new_name: nested_dictionary_get(d=tweet, key=nested_key)
                for nested_key, new_name in TWEET_DICT_KEY_MAPPING}
    return new_dict

In [40]:
def load_and_parse_twarc_file_to_list_of_tweets(path: str, parse_tweet_fxn: Callable) -> List:
    with open(path, 'rb') as f:
        return [parse_tweet_fxn(json.loads(line)) for line in f]

In [41]:
def load_twarc_json_file_to_dataframe(path: str, parse_tweet_fxn: Callable) -> pd.DataFrame:
    print(f"Loading data from {path}.")
    tweets_list = load_and_parse_twarc_file_to_list_of_tweets(path=path, parse_tweet_fxn=parse_and_reduce_tweet)
    tweet_df = pd.DataFrame.from_records(tweets_list)
    return tweet_df


def load_twarc_json_files_to_dataframe(paths: List[str], parse_tweet_fxn: Callable) -> pd.DataFrame:
    tweet_dfs = [load_twarc_json_file_to_dataframe(path, parse_tweet_fxn) for path in paths]
    tweet_df = pd.concat(tweet_dfs)
    return tweet_df

In [56]:
def save_tweets_df(df, path):
    print(f"Saving data to {path}.")
    df.to_parquet(path)

In [62]:
def load_tweets_df(path):
    print(f"Loading data from {path}.")
    return pd.read_parquet(path)

# 1. Load Tweets Data, Enrich with Sentiment, and Export

#### Load tweets to a dataframe

In [42]:
paths = [
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/Arsenal_Strngth.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/BeTitanFit.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/BodySolidFit.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/BodycraftFit.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/CAPBarbell84.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/HOISTFitness.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/HammerStrength.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/LifeFitness.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/PRxPerformance.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/Powertec.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/RepFitnessEquip.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/RogueFitness.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/Sorinex.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/TorqueFitness.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/bellsofsteel.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/concept2.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/cybex.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/elitefts.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/force_usa.jsonl",
    "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/onepeloton.jsonl"
]

In [43]:
# tweet_df = load_twarc_json_file_to_dataframe(path=paths[0], parse_tweet_fxn=parse_and_reduce_tweet)
tweet_df = load_twarc_json_files_to_dataframe(paths=paths, parse_tweet_fxn=parse_and_reduce_tweet)

Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/Arsenal_Strngth.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/BeTitanFit.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/BodySolidFit.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/BodycraftFit.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/CAPBarbell84.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/HOISTFitness.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/HammerStrength.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/LifeFitness.jsonl.
Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/PRxPerformance.jsonl.
Loading data from /Users/b.feifke

#### Validate Data

In [44]:
tweet_df.head().T

,0,1,2,3,4
url,https://twitter.com/Ayoobi77/status/1567340537333837824,https://twitter.com/Arsenal_Strngth/status/1567145753360977922,https://twitter.com/Arsenal_Strngth/status/1567144502334308354,https://twitter.com/Ayoobi77/status/1566312069196124166,https://twitter.com/Ayoobi77/status/1566311005038297089
content,@Arsenal_Strngth The true face of sports engineering 🙂\nArsenal Strength,Shipping soon: black on black Reloaded Iso Lat Pull down. https://t.co/GgnZl99ZhI,@Ayoobi77 @Collin_Abel10 @EaglevilleFB @CoachFWalker Thank you! ⚔️,@Collin_Abel10 @Arsenal_Strngth @EaglevilleFB @CoachFWalker The best machine,@Arsenal_Strngth Excellent products from Arsenal Strength 👍👍👍
tweet_id,1567340537333837824,1567145753360977922,1567144502334308354,1566312069196124166,1566311005038297089
user_id,787689201231888384,2942005261,2942005261,787689201231888384,787689201231888384
user_name,Ayoobi77,Arsenal_Strngth,Arsenal_Strngth,Ayoobi77,Ayoobi77
datetime,2022-09-07T02:34:38+00:00,2022-09-06T13:40:38+00:00,2022-09-06T13:35:40+00:00,2022-09-04T06:27:52+00:00,2022-09-04T06:23:38+00:00


In [45]:
tweet_df.shape

(832517, 6)

In [47]:
tweet_df.isnull().mean()

url          0.0
content      0.0
tweet_id     0.0
user_id      0.0
user_name    0.0
datetime     0.0
dtype: float64

#### Create Sample dataframe for testing

In [48]:
tweet_df_sample = tweet_df.sample(1000)

#### Enrich Tweets with sentiment

From [Vader docs](https://vadersentiment.readthedocs.io/en/latest/pages/about_the_scoring.html):  
> The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a ‘normalized, weighted composite score’ is accurate.

In [49]:
analyzer = SentimentIntensityAnalyzer()

In [50]:
tweet_df["sentiment"] = tweet_df["content"].apply(lambda x: analyzer.polarity_scores(x)["compound"])

In [51]:
tweet_df.head().T

,0,1,2,3,4
url,https://twitter.com/Ayoobi77/status/1567340537333837824,https://twitter.com/Arsenal_Strngth/status/1567145753360977922,https://twitter.com/Arsenal_Strngth/status/1567144502334308354,https://twitter.com/Ayoobi77/status/1566312069196124166,https://twitter.com/Ayoobi77/status/1566311005038297089
content,@Arsenal_Strngth The true face of sports engineering 🙂\nArsenal Strength,Shipping soon: black on black Reloaded Iso Lat Pull down. https://t.co/GgnZl99ZhI,@Ayoobi77 @Collin_Abel10 @EaglevilleFB @CoachFWalker Thank you! ⚔️,@Collin_Abel10 @Arsenal_Strngth @EaglevilleFB @CoachFWalker The best machine,@Arsenal_Strngth Excellent products from Arsenal Strength 👍👍👍
tweet_id,1567340537333837824,1567145753360977922,1567144502334308354,1566312069196124166,1566311005038297089
user_id,787689201231888384,2942005261,2942005261,787689201231888384,787689201231888384
user_name,Ayoobi77,Arsenal_Strngth,Arsenal_Strngth,Ayoobi77,Ayoobi77
datetime,2022-09-07T02:34:38+00:00,2022-09-06T13:40:38+00:00,2022-09-06T13:35:40+00:00,2022-09-04T06:27:52+00:00,2022-09-04T06:23:38+00:00
sentiment,0.8275,0.0,0.4199,0.6369,0.7845


#### Validate sentiment

In [55]:
tweet_df[["content", "sentiment"]].head(20)

,content,sentiment
0,@Arsenal_Strngth The true face of sports engineering 🙂\nArsenal Strength,0.8275
1,Shipping soon: black on black Reloaded Iso Lat Pull down. https://t.co/GgnZl99ZhI,0.0000
2,@Ayoobi77 @Collin_Abel10 @EaglevilleFB @CoachFWalker Thank you! ⚔️,0.4199
3,@Collin_Abel10 @Arsenal_Strngth @EaglevilleFB @CoachFWalker The best machine,0.6369
4,@Arsenal_Strngth Excellent products from Arsenal Strength 👍👍👍,0.7845
5,@Arsenal_Strngth Thanks 🙂,0.6815
6,"@Ayoobi77 Hello! The dimensions for this piece are Width: 37.256’’, Height: 45’’, Length 71’’",0.0000
7,@Arsenal_Strngth Kindly share the dimensions of alpha bent over row. Please,0.7717
8,All hail the king of leg presses. #arsenalstrength #legday https://t.co/qS0hH1w6UG,0.0772
9,@Collin_Abel10 @EaglevilleFB @CoachFWalker @Arsenal_Strngth Looks amazing!! I’m sure the boys are very thankful and can’t wait to start using the new equipment!!,0.9054


Sentiment seems reasonable.

#### Export tweets df

In [59]:
path = "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/tweet_df.parquet"
save_tweets_df(tweet_df, path)

Saving data to /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/tweet_df.parquet.


# 2. Analyze

In [63]:
path = "/Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/tweet_df.parquet"
tweet_df = load_tweets_df(path)

Loading data from /Users/b.feifke/code/home-gym-vendor-twitter-sentiment-analysis/data/tweet_df.parquet.
